# Heat equation

## imports

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from IPython.display import HTML
from scipy import special
import math

## initialization

In [10]:
X_max = 100
Y_max = 120
X_min = Y_min = 0

NUMBER_X = 1000
NUMBER_Y = 1000

In [3]:
X_RANGE = np.linspace(X_min, X_max, NUMBER_X)
Y_RANGE = np.linspace(Y_min, Y_max, NUMBER_Y)

X_STEP = X_RANGE[1] - X_RANGE[0]
Y_STEP = Y_RANGE[1] - Y_RANGE[0]

In [11]:
X, Y = np.meshgrid(X_RANGE, Y_RANGE, indexing='ij')

T = np.zeros_like(X)
lambdas = np.ones_like(X)
source_c = np.zeros_like(X)
source_p = np.zeros_like(X)

In [22]:
def calculate_coefficients(u, u_upper, u_lower, s, kind):
    """makes Thomas algorithm for longituginal and transversal kinds """
    if kind == 'longituginal':
        h_x = X_STEP
        h_y = Y_STEP
    elif kind == 'transversal':
        h_x = Y_STEP
        h_y = X_STEP
    
    a = b = h_y ** 2
    c = 2 * (h_x ** 2 + h_y ** 2)
    d = h_x ** 2 * (np.roll(u, -1) + np.roll(u, 1)) - s
    
    for i in range(1, len(u)-1):
        alpha[i+1] = b / (c - alpha[i] * a)
        beta[i+1] = (a * beta[i] + d) / (c - alpha[i] * a)
        
    return alpha,beta
    
def thomas_algorithm(u, u_upper, u_lower, s, kind):
    alpha, beta = calculate_coefficients(u, u_upper, u_lower, s, kind)
    # get from alpha and beta u

## boundary conditions

<img src="task3.png" height="100px" width="400px" >

In [18]:
lambda_edge = 10 ** 15
T_edge = 150

### 1

In [19]:
lambdas[0:10,0:10] = lambda_edge
T[0:10,0] = T_edge
T[0,0:10] = T_edge

## main procedure

In [20]:
check = True

In [ ]:
# while check:
    